In [1]:
from pylab import *
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)        
plt.rcParams['image.interpolation'] = 'nearest'  
plt.rcParams['image.cmap'] = 'gray'  
import sys
caffe_root = '../../'
sys.path.insert(0, caffe_root + 'python')
import caffe

../../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Net<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
../../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Blob<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
../../python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Solver<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \


In [2]:
##### exp parameter setting
# ER2ID+
model_def = caffe_root + 'yyp/JAFER/train_val.prototxt'
model_weights = caffe_root + 'yyp/JAFER/jaffe_er.caffemodel'
target_solver =  caffe_root + 'yyp/JAFID/ftsolver.prototxt'

#ID2ER+
#model_def = caffe_root + 'yyp/JAFID/train_val.prototxt'
#model_weights = caffe_root + 'yyp/JAFID/jaffe_id.caffemodel'
#target_solver =  caffe_root + 'yyp/JAFER/ftsolver.prototxt'

#ID2ID+
#model_def = caffe_root + 'yyp/JAFID/train_val2.prototxt'
#model_weights = caffe_root + 'yyp/JAFID/jaffe_id.caffemodel'
#target_solver =  caffe_root + 'yyp/JAFID/ftsolver.prototxt'

#ER2ER+
#model_def = caffe_root + 'yyp/JAFER/train_val2.prototxt'
#model_weights = caffe_root + 'yyp/JAFER/jaffe_er.caffemodel'
#target_solver =  caffe_root + 'yyp/JAFER/ftsolver.prototxt'

In [3]:
###### function defination
# polt train loss
def visLoss(niter,train_loss):
    _, ax1 = subplots()
    ax1.plot(arange(niter), train_loss)
    ax1.set_xlabel('iteration')
    ax1.set_ylabel('train loss')
# train
def trainNet(solver,niter,dis_interval):
    train_loss = zeros(niter)
    for it in range(niter):
        solver.step(1)
        train_loss[it] = solver.net.blobs['loss'].data
        if it % dis_interval == 0:
            print 'Iteration', it, 'loss...'
            print train_loss[it]
    return train_loss

In [4]:
np.random.seed(42)
caffe.set_device(0)
caffe.set_mode_gpu()
#####  load source net model
SourceNet = caffe.Net(model_def, model_weights, caffe.TRAIN)
print 'source net structure'
for layer_name, param in SourceNet.params.iteritems():
    print layer_name + '\t' + str(param[0].data.shape), str(param[1].data.shape)
#####  create target net solver
solver = caffe.SGDSolver( target_solver)
print 'target net structure'
for layer_name, param in solver.net.params.iteritems():
    print layer_name + '\t' + str(param[0].data.shape), str(param[1].data.shape)

source net structure
conv1	(20, 3, 5, 5) (20,)
conv2	(50, 20, 5, 5) (50,)
ip1	(500, 8450) (500,)
ip2	(100, 500) (100,)
target net structure
conv1	(20, 3, 5, 5) (20,)
conv2	(50, 20, 5, 5) (50,)
ip1	(500, 8450) (500,)
ip2	(100, 500) (100,)


In [5]:
#solver.net.copy_from(model_weights);
# transfer conv1 conv2
solver.net.params['conv1'][:] = SourceNet.params['conv1'][:]
#solver.net.params['conv2'][:] = SourceNet.params['conv2'][:]
##### test transferred net
solver.net.forward() # train net

{'loss': array(51.62632751464844, dtype=float32)}

In [6]:
#niter = 101;
#dis_interval = 10;
#train_loss = trainNet(solver,niter,dis_interval)
#visLoss(niter,train_loss)
solver.solve();